In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [233]:
bnf_1min = pd.read_csv("../data/sensex_wave.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [234]:
bnf_1min.head()

,time,open,high,low,close,MA,Plot,Zero line,MACD Signal,datetime
3724,2017-01-02T09:15:00+05:30,26717.8100,26720.9800,26484.5300,26516.6300,26183.0445,216.1595,0,NaN,2017-01-02 09:15:00+05:30
3725,2017-01-02T10:15:00+05:30,26516.3500,26532.0700,26447.0600,26490.4900,26186.5105,179.4486,0,NaN,2017-01-02 10:15:00+05:30
3726,2017-01-02T11:15:00+05:30,26492.3400,26527.2200,26460.9900,26496.3600,26189.9963,154.2724,0,NaN,2017-01-02 11:15:00+05:30
3727,2017-01-02T12:15:00+05:30,26496.1800,26540.5700,26477.1000,26538.4500,26193.4997,146.5679,0,NaN,2017-01-02 12:15:00+05:30
3728,2017-01-02T13:15:00+05:30,26538.1800,26659.6400,26529.5700,26649.8000,26198.5518,168.5438,0,168.5438,2017-01-02 13:15:00+05:30


In [235]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [236]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2020, 8, 7),
 datetime.date(2023, 3, 28),
 datetime.date(2018, 11, 13),
 datetime.date(2019, 5, 27),
 datetime.date(2023, 2, 23),
 datetime.date(2021, 5, 31),
 datetime.date(2021, 7, 29),
 datetime.date(2022, 10, 14),
 datetime.date(2022, 8, 18),
 datetime.date(2018, 2, 19),
 datetime.date(2017, 4, 19),
 datetime.date(2023, 8, 11),
 datetime.date(2019, 8, 19),
 datetime.date(2018, 1, 11),
 datetime.date(2022, 6, 2),
 datetime.date(2022, 6, 7),
 datetime.date(2023, 3, 22),
 datetime.date(2024, 8, 20),
 datetime.date(2020, 9, 4),
 datetime.date(2024, 3, 27),
 datetime.date(2022, 1, 18),
 datetime.date(2022, 8, 24),
 datetime.date(2020, 9, 30),
 datetime.date(2017, 3, 10),
 datetime.date(2020, 2, 12),
 datetime.date(2021, 7, 9),
 datetime.date(2017, 11, 15),
 datetime.date(2023, 5, 15),
 datetime.date(2018, 2, 21),
 datetime.date(2018, 4, 24),
 datetime.date(2022, 3, 29),
 datetime.date(2017, 8, 1),
 datetime.date(2017, 5, 10),
 datetime.date(2020, 7, 28),
 datetime.date(20

In [237]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [259]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0001
LEVERAGE_ = 5

In [260]:
def calculate_signals(df, ema_length, x_days, pct):
    # Calculate X-day High/Low and 20 EMA
    df['X_High'] = df['high'].rolling(x_days).mean()
    df['X_Low'] = df['low'].rolling(x_days).mean()
    df['EMA_20'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Trend conditions
    df['EMA_Trend'] = df['EMA_20'] > df['EMA_20'].shift(1)
    
    # Signal conditions
    df['Buy_Signal'] = (
        (df['close'] > df['X_High'].shift(1) * (1 + (pct / 100))) & 
        (df['EMA_Trend']) & 
        (df['close'] > df['EMA_20'])
    )
    df['Sell_Signal'] = (
        (df['close'] < df['X_Low'].shift(1) * (1 - (pct / 100))) & 
        (~df['EMA_Trend']) & 
        (df['close'] < df['EMA_20'])
    )
    return df

In [261]:
# def backtest(df):
#     position = 0
#     entry_price = 0
#     trades = []
#     trailing_stop = None
    
#     for i in range(len(df)):
#         if df.loc[i, 'Buy_Signal'] and position == 0:
#             position = 1
#             # entry_price = df.loc[i, 'close']
#             trailing_stop_long = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Buy', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_long = df.loc[i, 'datetime']
#             entry_price_long = df.loc[i, 'close']
#             initial_sl_long = trailing_stop_long
        
#         elif df.loc[i, 'Sell_Signal'] and position == 0:
#             position = -1
#             entry_price = df.loc[i, 'close']
#             trailing_stop_short = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Sell', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_short = df.loc[i, 'datetime']
#             entry_price_short = df.loc[i, 'close']
#             initial_sl_short = trailing_stop_short
        
#         elif position == 1:
#             if df.loc[i, 'low'] < trailing_stop_long:
#                 # trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#                 exit_time_long = df.loc[i, 'datetime']
#                 exit_price_long = trailing_stop_long
#                 points_long = exit_price_long - entry_price_long
                
#             elif df.loc[i, 'Add_Long']:
#                 # trades.append({'Type': 'Add Long', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 entry_time_long = df.loc[i, 'datetime']
#                 entry_price_long = df.loc[i, 'close']
#                 initial_sl_long = trailing_stop_long
#                 trailing_stop_long = max(trailing_stop_long, df.loc[i, 'EMA_20'])
        
#         elif position == -1:
#             if df.loc[i, 'high'] > trailing_stop:
#                 trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#             elif df.loc[i, 'Add_Short']:
#                 trades.append({'Type': 'Add Short', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 trailing_stop = min(trailing_stop, df.loc[i, 'EMA_20'])
    
#     return pd.DataFrame(trades)

In [262]:
def backtest(df):
    # print(df.to_string())
    # return
    df.reset_index(drop=True, inplace=True)
    # Variables for long trades
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    tradebook_long = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_long = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for long trades
        long_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i, 'close']
            long_entry_date = df.loc[i, 'datetime']
            long_trailing_stop = df.loc[i, 'EMA_20']
            long_initial_sl = df.loc[i, 'X_Low']

            tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'Long',
                'Entry_Price': df.loc[i, 'close'],
                'Previous MA Value': df.loc[i-1, 'X_High'],
                'Initial SL': long_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = long_initial_sl
                lock_initial_sl = True

            can_pyramid = True
            pyramid_number = 0
            pyramid_high = 0
            continue
                
        # Exit or add to long position
        if long_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'low'] <= long_initial_sl:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , long_initial_sl , 'Initial SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0
            
            elif df.loc[i, 'close'] <= long_trailing_stop:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            # elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
            #     # Previous Y Candle Low Breached, confirmation found. Add position on high break
            #     can_add_long = True

            # if can_add_long and long_position and can_pyramid:
            #     if df.loc[i, 'high'] >= y_days_high and pyramid_number < pyr_num:
            #         tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
            #             'Trade No.': trade_number,
            #             'Entry_Date': df.loc[i, 'datetime'],
            #             'Exit_Date': None,
            #             'Trade_Type': 'Add_Long',
            #             'Entry_Price': y_days_high,
            #             'Initial SL': first_sl,
            #             'Exit_Price': None,
            #             # 'Profit/Loss': None  # Filled when exited
            #         }])], ignore_index=True)
            #         can_add_long = False
            #         pyramid_number += 1
            #         can_pyramid = True
    
    tradebook = pd.concat([tradebook_long, tradebook], ignore_index=True)
    return tradebook


In [263]:
# def backtest(df, y_days):
#     # Variables for long trades
#     long_position = 0  # 0 = no position, 1 = long
#     long_entry_price = 0
#     long_entry_date = None
#     long_trades = []
#     tradebook = pd.DataFrame()
#     tradebook_long = pd.DataFrame()
#     tradebook_short = pd.DataFrame()
#     long_trailing_stop = None
#     # Variables for short trades
#     short_position = 0  # 0 = no position, -1 = short
#     short_entry_price = 0
#     short_entry_date = None
#     short_trades = []
#     short_trailing_stop = None

#     can_add_long = False
#     can_add_short = False

#     trade_number = 0

#     for i in range(len(df)):
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']
        
#         if df.loc[i, 'Buy_Signal'] and long_position == 0:
#             long_position = 1
#             long_entry_price = df.loc[i, 'close']
#             long_entry_date = df.loc[i, 'datetime']
#             long_trailing_stop = df.loc[i, 'EMA_20']
#             long_initial_sl = df.loc[i, 'X_Low']

#             tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Long',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': long_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         # Exit or add to long position
#         if long_position == 1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'low'] <= long_initial_sl:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , long_initial_sl]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] <= long_trailing_stop:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1

#             elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
#                 # Previous Y Candle Low Breached, confirmation found. Add position on high break
#                 can_add_long = True

#             if can_add_long and long_position:
#                 if df.loc[i, 'high'] >= y_days_high:
#                     tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Long',
#                         'Entry_Price': y_days_high,
#                         'Initial SL': long_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_long = False

#     for i in range(len(df)):
        
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']

#         # Entry signals for short trades
#         if df.loc[i, 'Sell_Signal'] and short_position == 0:
#             short_position = -1
#             short_entry_price = df.loc[i, 'close']
#             short_entry_date = df.loc[i, 'datetime']
#             short_trailing_stop = df.loc[i, 'EMA_20']
#             short_initial_sl = df.loc[i, 'X_High']

#             tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Short',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': short_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         if short_position == -1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'high'] >= short_initial_sl:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] ,short_initial_sl]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] >= short_trailing_stop:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1

#             elif (df.loc[i, 'high'] >= y_days_high) and not can_add_short:
#                 # Previous Y Candle High Breached, confirmation found. Add position on low break
#                 can_add_short = True

#             if can_add_short and short_position:
#                 if df.loc[i, 'low'] <= y_days_low:
#                     tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Short',
#                         'Entry_Price': y_days_low,
#                         'Initial SL': short_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_short = False
    
#     tradebook = pd.concat([tradebook_long, tradebook_short], ignore_index=True)
#     return tradebook


In [264]:
ema_length = 55
x_days = 9
# y_days = 4
pct = 0.5
RPT = 5
df = bnf_1min
df = calculate_signals(df, ema_length,x_days, pct)
# print(df.tail().to_string())
# df1 = df[df['Buy_Signal']]
# print(df1.to_string())
tb = backtest(df)
tb = tb.sort_values(by='Entry_Date')
# variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_48125/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


In [265]:
import numpy as np
portfolio = 10000000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
# tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
tb['Qty'] = portfolio * 8 / tb['Entry_Price']
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
tb['Trade Year'] = tb['Entry_Date'].dt.year

In [266]:
# tb['Cumulative_PnL'] = tb['PnL w cs'].cumsum()

In [267]:
# tb.to_csv('GOLD TRADE CHECK JJPP.csv', index=False)

In [268]:
tb = tb.reset_index()
# tb

In [269]:
import matplotlib.pyplot as plt

def plot_cumulative_pnl(tb, title="Cumulative PnL Over Time"):
    if 'Cumulative_PnL' not in tb.columns:
        raise ValueError("The DataFrame must contain a 'Cumulative_PnL' column.")

    # Plot cumulative PnL
    plt.figure(figsize=(12, 3))
    plt.plot(tb.index, tb['Cumulative_PnL'], label='Cumulative PnL', color='blue', linewidth=2)
    plt.title(title, fontsize=8)
    plt.xlabel('Trade Index', fontsize=6)
    plt.ylabel('Cumulative PnL', fontsize=6)
    plt.legend(fontsize=6)
    plt.grid(alpha=0.5)
    plt.show()


In [270]:
# tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
# tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()

In [271]:
# tb_long_only = tb_long_only[tb_long_only['Trade Year'] >= 2017]

In [272]:
# plot_cumulative_pnl(tb_long_only)
# print(tb_long_only['PnL w cs'].sum())

In [273]:
# plot_cumulative_pnl(tb_short_only)
# print(tb_short_only['PnL w cs'].sum())

In [274]:
# plot_cumulative_pnl(tb)
# print(tb['PnL w cs'].sum())

In [275]:
# plot_cumulative_pnl(tb2)
# print(tb2['PnL w cs'].sum())

In [276]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [277]:
tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
tb['Trade Year'] = tb['Entry_Date'].dt.year
# tb['ROI% w cs'] = tb['ROI%']

In [278]:
stats = generate_stats(tb, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,76.9392,21,52.3810,11.5031,-4.9594,-22.5814,3.4072,...
2018,106.1714,23,39.1304,16.9441,-3.3089,-19.6744,5.3964,...
2019,40.3755,29,34.4828,13.6922,-5.0814,-23.9593,1.6852,...
2020,266.9010,31,41.9355,29.7299,-6.6437,-39.3391,6.7846,...
2021,145.8137,29,27.5862,29.7870,-4.4039,-47.1156,3.0948,...
2022,69.4994,29,27.5862,22.7056,-5.3402,-45.4130,1.5304,...
2023,114.6783,20,40.0000,21.2529,-4.6121,-29.0234,3.9512,...
2024,62.3941,23,39.1304,14.7532,-5.4142,-23.0585,2.7059,...
Overall,882.7726,205,37.0732,20.0682,-5.0188,-47.1156,18.7363,...


# Gold 4H X_Days High, Y_Days Low 

In [279]:
tb1 = tb[tb['Trade_Type'] == 'Long']
tb2 = tb[tb['Trade_Type'] == 'Add_Long']
tb1['ROI%'].sum() , tb2['ROI%'].sum()

(882.7725518465887, 0)

In [281]:
tb.tail(25)

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Previous MA Value,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
180,180,180,2023-11-03 09:15:00+05:30,2023-12-20 14:15:00+05:30,Long,64493.7000,63997.8500,63947.5322,70494.5300,Trailing SL Hit,6000.8300,13.4988,5987.3312,1240.4312,7426872.6117,74.2687,2023
181,181,181,2023-12-22 09:15:00+05:30,2024-01-02 10:15:00+05:30,Long,71143.0500,70764.7544,70483.7589,71698.8000,Trailing SL Hit,555.7500,14.2842,541.4658,1124.4949,608875.5711,6.0888,2023
182,182,182,2024-01-12 09:15:00+05:30,2024-01-17 09:15:00+05:30,Long,72283.9700,71817.7067,71704.5244,72282.2200,Trailing SL Hit,-1.7500,14.4566,-16.2066,1106.7461,-17936.6119,-0.1794,2024
183,183,183,2024-01-29 09:15:00+05:30,2024-01-30 14:15:00+05:30,Long,71397.8800,70820.7911,70543.9900,71170.0700,Trailing SL Hit,-227.8100,14.2568,-242.0668,1120.4814,-271231.3531,-2.7123,2024
184,184,184,2024-02-02 09:15:00+05:30,2024-02-05 14:15:00+05:30,Long,72418.5000,71880.8878,71691.3678,71691.3678,Initial SL Hit,-727.1322,14.4110,-741.5432,1104.6901,-819175.4416,-8.1918,2024
185,185,185,2024-02-14 15:15:00+05:30,2024-02-22 09:15:00+05:30,Long,71833.7400,71458.6022,71233.7833,72243.6500,Trailing SL Hit,409.9100,14.4077,395.5023,1113.6828,440464.0616,4.4046,2024
186,186,186,2024-02-22 14:15:00+05:30,2024-02-28 10:15:00+05:30,Long,73137.9000,72730.0944,72363.9567,72677.6400,Trailing SL Hit,-460.2600,14.5816,-474.8416,1093.8241,-519393.1507,-5.1939,2024
187,187,187,2024-03-01 09:15:00+05:30,2024-03-11 14:15:00+05:30,Long,73204.1500,72534.7033,72288.6544,73499.3200,Trailing SL Hit,295.1700,14.6703,280.4997,1092.8342,306539.6189,3.0654,2024
188,188,188,2024-03-21 09:15:00+05:30,2024-03-21 11:15:00+05:30,Long,72845.6200,72193.1533,72022.3356,72447.3800,Trailing SL Hit,-398.2400,14.5293,-412.7693,1098.2129,-453308.5723,-4.5331,2024
189,189,189,2024-03-28 09:15:00+05:30,2024-04-12 13:15:00+05:30,Long,73594.9400,72957.8933,72866.5067,74326.5700,Trailing SL Hit,731.6300,14.7922,716.8378,1087.0313,779225.1467,7.7923,2024


In [108]:
tb.to_csv('SENSEX_jjpp_55_9_HALF_PCT_0_PYR.csv')

In [50]:
# tb_long_only.to_csv('GOLD Range JJPP.csv', index=False)

In [51]:
tb_long_only_1 = tb_long_only[tb_long_only['Trade_Type'] == 'Add_Long']
tb_long_only_2 = tb_long_only[tb_long_only['Trade_Type'] == 'Long']

In [52]:
tb_long_only_1['ROI%'].sum() , tb_long_only_2['ROI%'].sum()

(41.66300366972138, 194.59838974405739)

In [110]:
# ema_length = 55
# x_days = 28
# y_days = 7
# pct = 1.05
# RPT = 5

pct_range = [0.25, 0.5,0.75, 1]

for ema_length in range(25, 81, 5):
    for x_days in range(4, 37, 4):
            for pct in pct_range:
                # RPT = 5
                variation = f'EMA:{ema_length}, X:{x_days}, PCT:{pct}'
                print(variation)
                df = bnf_1min
                df = calculate_signals(df, ema_length=ema_length, x_days=x_days, pct=pct)
                tb = backtest(df)
                if len(tb) > 0:
                    tb = tb.sort_values(by='Entry_Date')
                    portfolio = 10000000
                    tb['PnL'] = np.where(
                        tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                        tb['Exit_Price'] - tb['Entry_Price'],
                        np.where(
                            tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
                            tb['Entry_Price'] - tb['Exit_Price'],
                            0  # Default case if Trade_Type is something unexpected
                        )
                    )
                    
                    tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                    tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                    tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                    tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                    tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                    tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                    tb['Trade Year'] = tb['Entry_Date'].dt.year
                    # tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                    # tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                    # tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                    # tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()

                    stats1 = generate_stats(tb, variation)
                    for x, y in stats1.items():
                        if x > 20:
                            # print('Long Only')
                            print(pd.DataFrame(y).to_string())
                        break

EMA:25, X:4, PCT:0.25


/tmp/ipykernel_48125/2724525255.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:25, X:4, PCT:0.5


/tmp/ipykernel_48125/2724525255.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:25, X:4, PCT:0.75


...

/tmp/ipykernel_48125/2724525255.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:0.75


/tmp/ipykernel_48125/2724525255.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:1


/tmp/ipykernel_48125/2724525255.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[
